In [231]:
import zipfile
import os
import xlwings as xw
import psutil
import os
import shutil
import zipfile
from pathlib import Path


not_table_file_types = ['pdf', 'jpeg', 'jpg', 'png', 'zip', 'PDF', 'JPEG', 'JPG', 'PNG', 'ZIP']
arch_file_types = ['zip', 'ZIP']
table_file_types = ['xlsx', 'xlsm', 'xls', 'ods', 'XLSX', 'XLSM', 'XLS', 'ODS']

current_dir = os.getcwd()+'\\'
main_dir = str(Path(os.getcwd()).parent)+'\\'
current_dir, main_dir

('C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\Nextcloud_connector\\',
 'C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\')

In [232]:
########### done modules #################################
import os
import shutil
import zipfile

########### DEF unpack_zipfile #################################
def unpack_zipfile(dir_with_archive, extract_dir='unzip', encoding='cp866'):
    for file in os.listdir(dir_with_archive):
        if '.zip' in file:
            with zipfile.ZipFile(dir_with_archive+file) as archive:
                for entry in archive.infolist():
                    try:
                        name = entry.filename.encode('cp437').decode(encoding)
                    except:
                        name = entry.filename.encode('cp866').decode('cp866')
                    target = os.path.join(extract_dir, *name.split('/'))    
                    os.makedirs(os.path.dirname(target), exist_ok=True)
                    if not entry.is_dir():  # file
                        with archive.open(entry) as source, open(target, 'wb') as dest:
                            shutil.copyfileobj(source, dest)



In [233]:
########### RUN unpack_zipfile #################################
unpack_zipfile(main_dir)

In [259]:
# формирование словаря, где ключ = путь_к_паке, а значения = список имено файлов в папке

unpacked_dict = {}
unpacked_dir_tree = os.walk(current_dir+'unzip'+'\\')


for i in unpacked_dir_tree:
    dir_structure = list(i)
    if dir_structure[-1]:
        unpacked_dict[dir_structure[0]+'\\'] = dir_structure[-1]

In [279]:
# unpacked_dict

In [276]:
# поиск архивов в папке unzip

for k,v in unpacked_dict.items():
    res = [string for string in v if '.zip' in string]
    for _ in res:
        match = os.path.join(k, _)
print(match)

C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\Nextcloud_connector\unzip\Контроль МО\КЖЗ.zip


In [262]:
# поиск папок КЖЗ (и содержимого этих папок) в папке unzip

for k,v in unpacked_dict.items():
    if 'КЖЗ' in k:
        print(k,v,sep='\n\n')

C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\Nextcloud_connector\unzip\Контроль МО\20.05.2022\КЖЗ 19.05.2022\19.05.2022\

['19.05.2022_ГП_контроль_МО_____КЖЗ.xlsx', 'игнатова.pdf', 'москаленко.pdf', 'некрасова.pdf', 'новикова.pdf', 'терехина.pdf', 'юсупова.pdf']
C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\Nextcloud_connector\unzip\Контроль МО\23.05.2022\КЖЗ\20.05.2022\

['20.05.2022_ГП_Контроль_МО_____КЖЗ.xlsx', 'ильина.pdf', 'ким.pdf', 'коротаева.pdf', 'некрасова.pdf', 'семанина.pdf', 'шмагина.pdf']


to do

1. распаковать архивы найденные в папках
2. маркировать файлы КЖЗ
2. сортировка файлов по папкам по расширению и содержимому таблиц


In [186]:
source = current_dir+'unzip'+'\\'
for _ in os.listdir(source):
    shutil.move(source+'\\'+_, 'unpack') 

Error: Destination path 'C:\Users\KorneevSN\Downloads\КНИГИ\REPO\Nextcloud_connector\Nextcloud_connector\unpacked\Контроль МО' already exists

In [180]:
########### done modules #################################
import os
import shutil
import zipfile

########### DEF unpack_zipfile #################################
def unpack_zipfile(filename, extract_dir, encoding='cp866'):
    with zipfile.ZipFile(filename) as archive:
        for entry in archive.infolist():
            try:
                name = entry.filename.encode('cp437').decode(encoding)
            except:
                name = entry.filename.encode('cp866').decode('cp866')
            if '.zip' in name:
                print('entry=',entry, 'name=', name, sep='\n')
                nested_archive = archive.getinfo(entry.filename)
                path_nested_archive = os.path.abspath(nested_archive.filename)              
                zipfile.ZipFile(path_nested_archive).extract(path_nested_archive, 'unpacked')
                print('nested_archive=',nested_archive, 'path_nested_archive=', path_nested_archive, sep='\n')
#                 with zipfile.ZipFile(filename) as archive:
#             else:          
#                 pass



#   docx_zip = zipfile.ZipFile(zip_path)
# docx_info = docx_zip.getinfo(name_of_image_in_zip)
# path_to_image = os.path.abspath(docx_info.filename)          
            
            

#                 # don't extract absolute paths or ones with .. in them
#                 if name.startswith('/'):
#                     print('dont extract',name)

#                 target = os.path.join(extract_dir, *name.split('/'))    
#                 os.makedirs(os.path.dirname(target), exist_ok=True)
#                 if not entry.is_dir():  # file
#                     with archive.open(entry) as source, open(target, 'wb') as dest:
#                         shutil.copyfileobj(source, dest)

########### RUN unpack_zipfile #################################
unpack_zipfile(main_dir+'23.05.2022-15.01-Контроль МО.zip', 'unpacked', encoding='cp866')

entry=
<ZipInfo filename='è«¡Γα«½∞ îÄ/èåç.zip' external_attr=0x20 file_size=738950>
name=
Контроль МО/КЖЗ.zip


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\Nextcloud_connector\\è«¡Γα«½∞ îÄ\\èåç.zip'

In [149]:
def old_unzip(file, unpack_dir, exceptions_unpack_dir):
    with zipfile.ZipFile(file) as zip:
    
        for zip_info in zip.infolist():
            if zip_info.filename[-1] == '/':
                continue
                try:
    #                 print(abs_path_str.encode('cp437').decode('cp866'))
                    name = zip_info.filename.encode('cp437').decode('cp866')
                except: 
    #                 print('ex== ',abs_path_str.encode('cp866').decode('cp866'))
                    name = zip_info.filename.encode('cp866').decode('cp866')
    
            if '.zip' in abs_path_str:
                with zipfile.ZipFile(abs_path_str) as nested_zip:
                    for nested_zip_info in nested_zip.infolist():
                        try:
            #                 print(abs_path_str.encode('cp437').decode('cp866'))
                            nested_abs_path_str = nested_zip_info.filename.encode('cp437').decode('cp866')
                        except: 
            #                 print('ex== ',abs_path_str.encode('cp866').decode('cp866'))
                            nested_abs_path_str = nested_zip_info.filename.encode('cp866').decode('cp866')
                    if '/КЖЗ' in abs_path_str and abs_path_str not in table_file_types:
                        abs_path_str = os.path.basename(abs_path_str)
                        zip.extract(zip_info, exceptions_unpack_dir)
                    elif abs_path_str[-1] == '/':
                        continue
                    name = os.path.basename(abs_path_str)
                    zip.extract(zip_info, unpack_dir)
                
            else:
                if '/КЖЗ' in abs_path_str and abs_path_str not in table_file_types:
                    abs_path_str = os.path.basename(abs_path_str)
                    zip.extract(zip_info, exceptions_unpack_dir)
                elif abs_path_str[-1] == '/':
                    continue
                
            name = os.path.basename(abs_path_str)
            zip.extract(zip_info, unpack_dir)

In [150]:
for _ in os.listdir(main_dir):
    if '.zip' in _:
        old_unzip(main_dir+_, 'old_unzip_script', 'exceptions_unpack_dir')

FileNotFoundError: [Errno 2] No such file or directory: 'Контроль МО/КЖЗ.zip'

In [64]:
########### subDEF unpack_zipfile #################################
def unpack_zipfile(filename,
                   extract_dir='unzip',
                   exceptions_dir='unzip_exceptions'
                   encoding='cp866'):
    with zipfile.ZipFile(filename) as archive:
        for entry in archive.infolist():
            try:
                name = entry.filename.encode('cp437').decode(encoding)  # reencode!!!
            except:
                name = entry.filename.encode('cp866').decode(encoding)  # reencode!!!
            print(name)

            # don't extract absolute paths or ones with .. in them
            if 'КЖЗ' in name:
                pass
#             if name.startswith('/') or '..' in name:
#                 continue

            target = os.path.join(extract_dir, *name.split('/'))    
            os.makedirs(os.path.dirname(target), exist_ok=True)
            if not entry.is_dir():  # file
                with archive.open(entry) as source, open(target, 'wb') as dest:
                    shutil.copyfileobj(source, dest)

In [280]:
# for _ in os.listdir(main_dir):
#     if '.zip' in _:
#         unpack_zipfile(main_dir + _)

In [61]:
########### DEF find_dir_by_name_and_extract_files #################################
def find_dir_by_name_and_extract_files(processing_dir_path,
                                       target_dir_name = '\КЖЗ\\',
                                       dir_for_replace = '',
                                       replacing_dir_name = '\КЖЗ\\'):
    """
    0) *dir_for_replace* - print path to parent folder for *replacing_dir_name* !!!
       default 'None' - *replacing_dir_name* will be replaced to *processing_dir_path*   
       
    1) Find subdir in *processing_dir_path* by *target_dir_name*,
    2) if need create *replacing_dir_name* as subdir in *processing_dir_path*
    2) replace files in *target_dir_name* to *replacing_dir_name* 
    """
    
#     os.chdir(processing_dir_path)
    for root, dirs, files in os.walk(os.getcwd(), topdown = False):
        for fld in dirs:
            abs_path_to_folder = os.path.join(root, fld)
            
            print(abs_path_to_folder)

            if target_dir_name in abs_path_to_folder:# and not_table_file_types in abs_path_to_folder:
                if dir_for_replace != '':
#                     print(dir_for_replace)
                    shutil.move(abs_path_to_folder, dir_for_replace)
                else: shutil.move(abs_path_to_folder, processing_dir_path +'\\'+ replacing_dir_name)

In [281]:
# for _ in os.listdir(current_dir + 'unzip\\'):
#     find_dir_by_name_and_extract_files(processing_dir_path = current_dir + 'unzip\\',
#                                        target_dir_name = 'КЖЗ',
#                                        dir_for_replace = current_dir + 'replace_kjz\\',
#                                        replacing_dir_name = 'КЖЗ')

In [71]:
def unzip(file, unpack_dir):
    with zipfile.ZipFile(file) as zip:
        for zip_info in zip.infolist():
            if 'КЖЗ' in zip_info.filename:
                pass
            if zip_info.filename[-1] == '/':
                continue
            zip_info.filename = os.path.basename(zip_info.filename)
            zip.extract(zip_info, unpack_dir)

In [282]:
# with zipfile.ZipFile(main_dir+'23.05.2022-15.01-Контроль МО.zip') as zip:
#     for zip_info in zip.infolist():
#         print(zip_info)

In [72]:
unzip(main_dir+'23.05.2022-15.01-Контроль МО.zip', 'old_arc')

In [64]:
########### DEF find zip file in parent dir and unpack_zipfile #################################
def get_archive_from_main_dir(main_dir):
    for _ in os.listdir(main_dir):
        if '.zip' in _:
            unpack_zipfile(main_dir+_)

In [65]:
get_archive_from_main_dir(main_dir)

In [66]:
unzip_folder = current_dir+'unzip\\'
unzip_folder

'C:\\Users\\KorneevSN\\Downloads\\КНИГИ\\REPO\\Nextcloud_connector\\Nextcloud_connector\\unzip\\'

In [67]:
########### DEF find_dir_by_name_and_extract_files #################################
def find_dir_by_name_and_extract_files(processing_dir_path,
                                       target_dir_name = 'КЖЗ',
                                       dir_for_replace = '',
                                       replacing_dir_name = 'КЖЗ'):
    """
    0) *dir_for_replace* - print path to parent folder for *replacing_dir_name* !!!
       default 'None' - *replacing_dir_name* will be replaced to *processing_dir_path*   
       
    1) Find subdir in *processing_dir_path* by *target_dir_name*,
    2) if need create *replacing_dir_name* as subdir in *processing_dir_path*
    2) replace files in *target_dir_name* to *replacing_dir_name* 
    """
    
#     os.chdir(processing_dir_path)
    for root, dirs, files in os.walk(os.getcwd(), topdown = False):
        for fld in dirs:
            abs_path_to_folder = os.path.join(root, fld)
            
            print(abs_path_to_folder)

            if target_dir_name in abs_path_to_folder.split('\\')[-1] == target_dir_name:
                if dir_for_replace != '':
#                     print(dir_for_replace)
                    shutil.move(abs_path_to_folder, dir_for_replace)
                else: shutil.move(abs_path_to_folder, processing_dir_path +'\\'+ replacing_dir_name)

In [283]:
# ########### RUN find_dir_by_name_and_extract_files #################################
# find_dir_by_name_and_extract_files(unzip_folder)